In [1]:
pip install yellowbrick

Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import mlflow
import mlflow.sklearn

In [3]:
#Carregar a base de dados

credito = pd.read_csv('./Credit.csv')

In [4]:
#Visualização das cincos primeiras linhas
credito.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


In [4]:
#Transformar as colunas categóricas em núemricas através da função abaixo

for col in credito.columns:
    if credito[col].dtype == 'object':
        credito[col] = credito[col].astype('category').cat.codes
credito.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0


In [5]:
#Speração da base de dados entre treino e teste

previsores = credito.iloc[:,0:20].values
classe = credito.iloc[:,20].values

In [7]:
previsores

array([[ 2,  6,  1, ...,  1,  1,  1],
       [ 1, 48,  3, ...,  1,  0,  1],
       [ 0, 12,  1, ...,  2,  0,  1],
       ...,
       [ 0, 12,  3, ...,  1,  0,  1],
       [ 2, 45,  3, ...,  1,  1,  1],
       [ 1, 45,  1, ...,  1,  0,  1]], dtype=int64)

In [6]:
X_treinamento, X_test, y_treinamento, y_test = train_test_split(previsores, classe, test_size=0.3, random_state=123)

## Criar o experimento de MLOPS e exportar o modelo criado

In [25]:
from sklearn.metrics import confusion_matrix


In [30]:
mlflow.set_experiment("nb_primeiro")
with mlflow.start_run():
    naive_bayes = GaussianNB()
    naive_bayes.fit(X_treinamento, y_treinamento)
    previsoes = naive_bayes.predict(X_test)
    
    #metricas
    acuracia = accuracy_score(y_test, previsoes)
    recall = recall_score(y_test, previsoes)
    precision = precision_score(y_test, previsoes)
    f1 = f1_score(y_test, previsoes)
    auc = roc_auc_score(y_test, previsoes)
    log = log_loss(y_test, previsoes)
    
    #registrar as metricas
    mlflow.log_metric("acuracia", acuracia)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precisão", precision)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("log", log)
    
    
    #gráficos 
   # confusion = ConfusionMatrixDisplay(naive_bayes, X_test, y_test)
    #plt.savefig("confusion,png")
    #roc = plot_roc_curve(naive_bayes, X_test, y_test)
    #plt.savefig("roc.png")
    
    
    #Logar gráficos ao mlflow
    mlflow.log_artifact("confusion.png")
    mlflow.log_artifact("roc.png")
    
    #modelo
    mlflow.sklearn.log_model(naive_bayes, "ModeloNB")
    
    #informações da execução
    print("Modelo:", mlflow.active_run().info.run_uuid)

mlflow.end_run()

Modelo: fdf097429e904ea0bf325c19de13250e


C:\Users\BlueShift\AppData\Local\Programs\Python\Python310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
#Comanndo para ligar o meu notebook e o servidor para assim acompanhar as métricas do modelo criado

!mlflow ui 